# Scripted spectra

In [ ]:
using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, Interpolations
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

PATH = "/home/ubuntu/Archives/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

subdir = "small_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
T_final = 16000.
T_final = 32000.
τ_final = 1000.
τ_final = 2000.
# τ_final = 4000.
all_ωs = []
spec_sum = []

num_plots = 100
for instance_name in instance_names[1:num_plots]
    if occursin("results", instance_name)
        continue
    end
    try
        seed = match(pattern, instance_name)[1]    

        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")

        gap = λ[2, :] .- λ[1, :];
        exact_times = range(0, 1, 33)
        gaploc = exact_times[findfirst(x -> x == minimum(gap), gap)] 
        T_diags = T_final .* [gaploc - 0.1, gaploc - 0.05, gaploc]
        
        # for k in 1:1
        for k in 2:2
        # for k in 3:3
            push!(all_ωs, h5read(folder_name * "results_" *instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.3f/omegas", T_final, τ_final, T_diags[k] / T_final)))
            push!(spec_sum, h5read(folder_name * "results_" * instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.3f/data", T_final, τ_final, T_diags[k] / T_final)))
        end   
    catch
        printstyled(instance_name, "\n", color=:red)
    end
end

In [ ]:
num_plots = length(spec_sum)

In [ ]:
# map data to functions
spectral_sum_fs = []
for (k, (ωs, spec)) in enumerate(zip(all_ωs, spec_sum))
    push!(spectral_sum_fs, linear_interpolation(ωs, spec, extrapolation_bc=Line()))
end

In [ ]:
xlim_vals = (-0.05, 0.2)
new_ωs = range(xlim_vals..., 2001)
spectral_average = mean([map(spectral_sum_fs[k], new_ωs) for k in 1:length(spectral_sum_fs)]);

In [ ]:
figure(figsize=(4, 3))
subplot(111)
plot(new_ωs, spectral_average, "-k", ms=2)    
xlim(xlim_vals...)
ylim(-0., 0.1)
xlabel("\$\\nu\$")
ylabel("Ensemble-averaged spectrum", fontsize=12)
tight_layout()

In [ ]:
xlim_vals = (-0.05, 0.1)
fig, axs = subplots(figsize=(5, 50), nrows=num_plots, ncols=1)

for (k, (ωs, spec)) in enumerate(zip(all_ωs, spec_sum))
    ax = axs[k]     
    ax.plot(ωs, spec, "-")

    ax.set_xlim(xlim_vals...)
    ax.set_ylim(0, 0.05)  
    ax.set_xticks(range(xlim_vals..., 7))  
    ax.set_xticklabels([])
    # ax.set_ylabel(@sprintf("\$\\rho_{%i%i}(T, \\omega)\$", idx, idx))
    # ax.legend(frameon=false, handlelength=0, fontsize=12)
    if k%2 == 0
        ax.yaxis.tick_right()
    end
    ax.tick_params(axis="y", labelsize=10)
end
ax = subplot(num_plots, 1, num_plots)
ax.set_xlabel("\$\\nu\$")
ax.set_xticklabels(round.(range(xlim_vals..., 7), digits=3))

subplots_adjust(wspace=0.5, hspace=0.16)